In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [66]:
weather_df = pd.read_csv('output_data/weather_data.csv')

## Sort to view max Humidity value for map intensity
weather_df.sort_values(['Humidity %'],ascending=False)

,Country,City,Latitude,Long,Temperature(°F),Humidity %,Cloudiness %,Wind Speed(mph),Date
370,RU,Bilibino,68.0546,166.4372,-2.65,100,98,2.59,10/23/22 22:42:45
375,US,Juneau,58.3019,-134.4197,38.21,100,40,0.00,10/23/22 22:40:21
159,GL,Nuuk,64.1835,-51.7216,34.41,100,75,14.97,10/23/22 22:36:41
416,CA,High Level,58.5169,-117.1360,26.71,100,100,12.66,10/23/22 22:44:02
424,IR,Khorramshahr,30.4397,48.1664,69.76,100,5,2.30,10/23/22 22:44:16
...,...,...,...,...,...,...,...,...,...
130,MR,Adrar,20.5022,-10.0711,79.03,16,26,10.65,10/23/22 22:35:50
464,ML,Goundam,16.4145,-3.6708,80.53,15,81,10.54,10/23/22 22:45:24
59,ML,Araouane,18.9048,-3.5265,78.71,15,97,10.80,10/23/22 22:33:46
156,AU,Kununurra,-15.7667,128.7333,106.48,15,15,11.43,10/23/22 22:36:36


In [50]:
gmaps.configure(api_key=g_key)

In [51]:
## Format dataframe data for map 
locations = weather_df[ ['Latitude','Long'] ].astype(float)
weather_df_drop = weather_df.dropna()
humidity = weather_df['Humidity %'].astype(float)

In [53]:
## Create google map with heatmap layer
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 2)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

In [67]:
## Isolate the ideal conditions for snow. This is my ideal weather condition.
## Reduce the number of rows in the dataset as suggested.
## I chose to do this by country as the suggestion is 10 or fewer, and I have 9 unique countries in the ideal conditions.


ideal_weather_df = weather_df.loc[(weather_df['Temperature(°F)'] <= 32) & (weather_df['Humidity %'] >= 60)]
ideal_weather_df = ideal_weather_df.drop_duplicates(subset='Country')
ideal_weather_df

,Country,City,Latitude,Long,Temperature(°F),Humidity %,Cloudiness %,Wind Speed(mph),Date
9,CA,Tuktoyaktuk,69.4541,-133.0374,17.60,79,75,4.61,10/23/22 22:32:21
20,RU,Pevek,69.7008,170.3133,14.14,95,100,2.48,10/23/22 22:24:39
28,US,Barrow,71.2906,-156.7887,24.82,86,100,21.85,10/23/22 22:21:13
36,GL,Qaanaaq,77.4840,-69.3632,20.37,93,100,1.86,10/23/22 22:24:38
71,IS,Husavik,66.0449,-17.3389,31.55,77,100,8.61,10/23/22 22:34:07
121,KZ,Borovskoy,53.8000,64.1500,28.85,67,8,11.97,10/23/22 22:35:35
198,FI,Rovaniemi,66.5000,25.7167,29.61,80,40,8.05,10/23/22 22:37:48
226,SJ,Longyearbyen,78.2186,15.6401,22.84,74,75,8.05,10/23/22 22:38:38
429,NO,Karasjok,69.4719,25.5112,27.86,96,76,2.42,10/23/22 22:44:25


In [68]:
## Google Nearby Places api call loop. 

nearby_search_df = ideal_weather_df[['Country','City','Latitude','Long']].copy()

params = {
    "key": g_key,
    "radius": 5000,
    'types':'hotel',
    'keyword': 'hotel'
         }

for index,row in nearby_search_df.iterrows():
    
    lat = row['Latitude']
    long = row['Long']
    params['location'] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels_request = requests.get(base_url, params=params)
    
    hotels_request = hotels_request.json()
    
    try:
        nearby_search_df.loc[index, "hotel_name"] = hotels_request["results"][0]["name"]
        print(f"Found Hotel for: {row['City']}")
    except (KeyError, IndexError):
        print(f"Hotel not found in {row['City']}! Too far!")

Found Hotel for: Tuktoyaktuk
Found Hotel for: Pevek
Found Hotel for: Barrow
Found Hotel for: Qaanaaq
Found Hotel for: Husavik
Found Hotel for: Borovskoy
Found Hotel for: Rovaniemi
Found Hotel for: Longyearbyen
Found Hotel for: Karasjok


In [69]:
## Remove the NaN values that did not return a hotel within proximity. Reset the index for clarity. 
## Save as csv to avoid having to make another call on the same location data. 
hotel_df = nearby_search_df.dropna()
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.to_csv('output_data/hotel_data.csv',index=False)
hotel_df = pd.read_csv('output_data/hotel_data.csv')
hotel_df

,Country,City,Latitude,Long,hotel_name
0,CA,Tuktoyaktuk,69.4541,-133.0374,Smitty's Bed and Breakfast
1,RU,Pevek,69.7008,170.3133,Munitsipal'naya Gostinitsa
2,US,Barrow,71.2906,-156.7887,Top of the World Hotel
3,GL,Qaanaaq,77.4840,-69.3632,Qaanaaq Hotel
4,IS,Husavik,66.0449,-17.3389,Fosshotel Húsavík
5,KZ,Borovskoy,53.8000,64.1500,Marzhan
6,FI,Rovaniemi,66.5000,25.7167,Arctic City Hotel
7,SJ,Longyearbyen,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
8,NO,Karasjok,69.4719,25.5112,Scandic Karasjok


In [73]:
hotel_df

,Country,City,Latitude,Long,hotel_name
0,CA,Tuktoyaktuk,69.4541,-133.0374,Smitty's Bed and Breakfast
1,RU,Pevek,69.7008,170.3133,Munitsipal'naya Gostinitsa
2,US,Barrow,71.2906,-156.7887,Top of the World Hotel
3,GL,Qaanaaq,77.4840,-69.3632,Qaanaaq Hotel
4,IS,Husavik,66.0449,-17.3389,Fosshotel Húsavík
5,KZ,Borovskoy,53.8000,64.1500,Marzhan
6,FI,Rovaniemi,66.5000,25.7167,Arctic City Hotel
7,SJ,Longyearbyen,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
8,NO,Karasjok,69.4719,25.5112,Scandic Karasjok


In [72]:
## Code snippet provided by referential starter code notebook provided for this project. Resource providers credited in README.

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Long"]]

In [79]:
## Final map with marker layer and heatmap layer. 
fig = gmaps.figure()
markers = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))